<a href="https://colab.research.google.com/github/Bariharsh/Automated-Pneumonia-Detection/blob/main/deepvision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from datasets import load_dataset

ds = load_dataset("hf-vision/chest-xray-pneumonia")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00007.parquet:   0%|          | 0.00/446M [00:00<?, ?B/s]

data/train-00001-of-00007.parquet:   0%|          | 0.00/385M [00:00<?, ?B/s]

data/train-00002-of-00007.parquet:   0%|          | 0.00/68.9M [00:00<?, ?B/s]

...

data/validation-00000-of-00001.parquet:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/78.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5216 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/16 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/624 [00:00<?, ? examples/s]

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [6]:

IMG_SIZE = 224
BATCH_SIZE = 32

def preprocess(example):
    '''
    Preprocess the PIL image (python image library) into array numbers
    '''
    image = example["image"].convert("L")  # grayscale
    image = image.resize((IMG_SIZE, IMG_SIZE))
    image = np.array(image, dtype=np.float32) / 255.0
    image = np.expand_dims(image, axis=-1)  # (224,224,1)
    label = example["label"]
    return image, label

In [7]:
def hf_to_tf(dataset):
    '''
    it converts the hugging face datasets to tensor flow training pipeline
    '''
    return tf.data.Dataset.from_generator( # creates an tensorflow datasets by pulling dataa one item at a time in pythin generator
        lambda: (preprocess(x) for x in dataset),
        output_signature=(
            tf.TensorSpec(shape=(224,224,1), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int64)
        ) # output signature just gives the shapes and type of data that matches the resnet18 architecture
    )

train_ds = hf_to_tf(ds["train"]).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
# prefech:- it prepares the next batch while current batch being trained
# shuffle:- it mixes the training images randomly before feeding them to the model.
val_ds   = hf_to_tf(ds["validation"]).batch(BATCH_SIZE)
test_ds  = hf_to_tf(ds["test"]).batch(BATCH_SIZE)
#test

In [8]:
def residual_block(x, filters, stride=1):
    shortcut = x

    x = layers.Conv2D(filters, 3, strides=stride, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, 3, strides=1, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1, strides=stride, use_bias=False)(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)

    return x


In [9]:
def ResNet18(input_shape=(224,224,1), num_classes=2):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, 7, strides=2, padding="same", use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    # Conv2_x
    x = residual_block(x, 64)
    x = residual_block(x, 64)

    # Conv3_x
    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 128)

    # Conv4_x
    x = residual_block(x, 256, stride=2)
    x = residual_block(x, 256)

    # Conv5_x
    x = residual_block(x, 512, stride=2)
    x = residual_block(x, 512)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return models.Model(inputs, outputs)


In [10]:
model = ResNet18(num_classes=2)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 112, 112,  │      3,136 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 112, 112,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 112, 112,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 56, 56,    │          0 │ re_lu[0][0]       │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 56, 56,    │     36,864 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 56, 56,    │     36,864 │ re_lu_1[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 56, 56,    │          0 │ add[0][0]         │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 56, 56,    │     36,864 │ re_lu_2[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 56, 56,    │     36,864 │ re_lu_3[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_4[0][0]  

 Total params: 11,180,866 (42.65 MB)

 Trainable params: 11,171,266 (42.61 MB)

 Non-trainable params: 9,600 (37.50 KB)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
)


Epoch 1/10
    163/Unknown 614s 4s/step - accuracy: 0.8214 - loss: 0.5966

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


163/163 ━━━━━━━━━━━━━━━━━━━━ 615s 4s/step - accuracy: 0.8220 - loss: 0.5943 - val_accuracy: 0.5000 - val_loss: 3.0439
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 618s 4s/step - accuracy: 0.7919 - loss: 0.7870 - val_accuracy: 0.5000 - val_loss: 2.8272
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 618s 4s/step - accuracy: 0.8013 - loss: 0.6668 - val_accuracy: 0.5000 - val_loss: 1.5698
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 619s 4s/step - accuracy: 0.7939 - loss: 0.6147 - val_accuracy: 0.5000 - val_loss: 1.6108
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 618s 4s/step - accuracy: 0.7925 - loss: 0.6168 - val_accuracy: 0.5000 - val_loss: 1.2648
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 619s 4s/step - accuracy: 0.8059 - loss: 0.5399 - val_accuracy: 0.5000 - val_loss: 3.0805
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 616s 4s/step - accuracy: 0.7877 - loss: 0.5522 - val_accuracy: 0.5625 - val_loss: 0.9577
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 615s 4s/step - accuracy: 0.8129 - loss: 0.5388 - val_accuracy: 0.500

In [1]:
test_loss, test_acc = model.evaluate(test_ds)
print("Test Accuracy:", test_acc)


NameError: name 'model' is not defined